TF 2.0 Tutorial(DL Camp) - MNIST 입니다.

해당 문서는 TensorFlow 공식 Tutorial을 기반으로 수정하였습니다.



먼저 프로그램에 텐서플로 라이브러리를 임포트합니다:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try: 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

데이터셋을 로드하여 준비합니다.

공식적으로 제공하는 데이터셋은 [해당 링크](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) 에서 확인할 수 있습니다.

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

Dataset의 shape와 실제 값은 아래와 같습니다. 이미지의 각각의 pixel 값이라고 생각하시면 됩니다.

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train[0].reshape(28,28))

해당하는 Dataset의 shape와 이미지는 아래와 같습니다.

In [0]:
import matplotlib.pyplot as plt

INDEX = 0

plt.imshow(x_train[INDEX].reshape(28,28), cmap='gray_r')

print(x_train[INDEX].shape)
print(y_train[INDEX])


tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(32) 
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

케라스(Keras)의 [모델 서브클래싱(subclassing) API](https://www.tensorflow.org/guide/keras#model_subclassing)를 사용하여 `tf.keras` 모델을 만듭니다:

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.hidden_1 = Dense(15, activation='sigmoid')  
    self.output_layer = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    x = self.hidden_1(x) 
    return self.output_layer(x)

model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다.

사용할 수 있는 다른 [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)와 [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses) 함수는 해당 링크에서 확인할 수 있습니다.

현재는 MSE(Quadratic error) Loss 함수와 SGD Optimizer를 사용할 예정입니다.

In [0]:
loss_object = tf.keras.losses.MeanSquaredError()

optimizer = tf.keras.optimizers.SGD()

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델을 훈련합니다:

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images) 
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

이제 모델을 5 Epoch 동안 학습하면서, 테스트할 예정입니다.

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
train_ACC = []
train_LOSS = []
test_ACC = []
test_LOSS = []

EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch: {}, Loss: {}, ACC: {}, Test Loss: {}, Test ACC: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  
  train_ACC.append(train_accuracy.result()*100)
  train_LOSS.append(train_loss.result())
  test_ACC.append(test_accuracy.result()*100)
  test_LOSS.append(test_loss.result())

위와 같은 설정을 따라했다면 약 20%의 테스트 정확도를 달성합니다.


아래에는 학습과정 동안의 Train/Test Accuracy, Loss를 그래프로 확인할 수 있습니다.

In [0]:
plt.figure(figsize=(10,10))
plt.plot(range(1,EPOCHS+1), train_ACC, label="Train")
plt.plot(range(1,EPOCHS+1), test_ACC, label="Test")

plt.title("Accuracy Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=2)

plt.show()

In [0]:
plt.figure(figsize=(10,10))
plt.plot(range(1,EPOCHS+1), train_LOSS, label="Train")
plt.plot(range(1,EPOCHS+1), test_LOSS, label="Test")

plt.title("Loss Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=3)

plt.show()

아래에는 Model의 구조와 Parameter 수를 확인할 수 있습니다.

In [0]:
model.summary()

In [0]:
print((28*28*1)*15 + 15)    # Layer 1의 Weight + Bias
print(15*10 + 10)           # Layer 2의 Weight + Bias

# 실습 MISSION #1

20 epoch동안 돌려보고 테스트 정확도를 확인해봅니다.

# 실습 MISSION #2

* 첫 번째 히든 레이어 node 수 조정하기
* 두 번째 레이어 추가하기
* 하이퍼파라미터 조정하기 : Learning Rate (0.01, 0.1, 100.0)
